# Fetching Data from Oanda API Endpoints

Oanda API endpoints can be found at http://developer.oanda.com/rest-live-v20/account-ep/

In [35]:
import requests
import pandas as pd
import json
from dateutil import parser

In [2]:
account_data = pd.read_csv("~/Documents/temp/oanda/testnet_key.txt", sep=" ", header=None)
API_KEY = account_data[0][2]
ACCOUNT_ID = account_data[0][1]
OANDA_URL = account_data[0][0]

In [3]:
session = requests.Session()

## Fetching Data from the Candles Endpoint

In [4]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [5]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [6]:
def set_candles_url(url, pair):
 return f"{url}/instruments/{pair}/candles"

In [7]:
pair_name = "EUR_USD"
url_candles = set_candles_url(OANDA_URL, pair_name)

In [8]:
response = session.get(url_candles, params=params, data=None, headers=None)

In [9]:
response.status_code

200

In [10]:
response.json()

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 1562,
   'time': '2022-07-19T22:00:00.000000000Z',
   'bid': {'o': '1.02260', 'h': '1.02408', 'l': '1.02235', 'c': '1.02347'},
   'mid': {'o': '1.02283', 'h': '1.02416', 'l': '1.02273', 'c': '1.02356'},
   'ask': {'o': '1.02306', 'h': '1.02425', 'l': '1.02285', 'c': '1.02366'}},
  {'complete': True,
   'volume': 2066,
   'time': '2022-07-19T23:00:00.000000000Z',
   'bid': {'o': '1.02346', 'h': '1.02362', 'l': '1.02288', 'c': '1.02297'},
   'mid': {'o': '1.02356', 'h': '1.02370', 'l': '1.02298', 'c': '1.02306'},
   'ask': {'o': '1.02366', 'h': '1.02382', 'l': '1.02307', 'c': '1.02314'}},
  {'complete': True,
   'volume': 2933,
   'time': '2022-07-20T00:00:00.000000000Z',
   'bid': {'o': '1.02302', 'h': '1.02479', 'l': '1.02300', 'c': '1.02454'},
   'mid': {'o': '1.02310', 'h': '1.02488', 'l': '1.02309', 'c': '1.02462'},
   'ask': {'o': '1.02317', 'h': '1.02499', 'l': '1.02317', 'c': '1.02469'}},

## Fetching Data from the Instruments Endpoint

In [11]:
url_inst = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [12]:
response = session.get(url_inst, params=params, data=None, headers=None)

In [13]:
response.status_code

200

In [14]:
data = response.json() # dict_keys(['instruments', 'lastTransactionID'])

In [15]:
instruments_list = data["instruments"]

In [16]:
len(instruments_list)

129

In [17]:
instruments_list

[{'name': 'CHINAH_HKD',
  'type': 'CFD',
  'displayName': 'China H Shares',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'minimumTradeSize': '0.1',
  'maximumTrailingStopDistance': '10000.0',
  'minimumTrailingStopDistance': '5.0',
  'maximumPositionSize': '0',
  'maximumOrderUnits': '500',
  'marginRate': '0.1',
  'guaranteedStopLossOrderMode': 'DISABLED',
  'tags': [{'type': 'ASSET_CLASS', 'name': 'INDEX'}],
  'financing': {'longRate': '-0.0259',
   'shortRate': '-0.0241',
   'financingDaysOfWeek': [{'dayOfWeek': 'MONDAY', 'daysCharged': 1},
    {'dayOfWeek': 'TUESDAY', 'daysCharged': 1},
    {'dayOfWeek': 'WEDNESDAY', 'daysCharged': 1},
    {'dayOfWeek': 'THURSDAY', 'daysCharged': 1},
    {'dayOfWeek': 'FRIDAY', 'daysCharged': 3},
    {'dayOfWeek': 'SATURDAY', 'daysCharged': 0},
    {'dayOfWeek': 'SUNDAY', 'daysCharged': 0}]}},
 {'name': 'AU200_AUD',
  'type': 'CFD',
  'displayName': 'Australia 200',
  'pipLocation': 0,
  'displayPrecision': 1,
  'trade

In [18]:
# taking a look at the information that's available for every instrument
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [19]:
# list of keys we're intersted in getting 
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 
         'tradeUnitsPrecision', 'marginRate']

In [20]:
instruments_dict = {}
for i in instruments_list:
    # print(i['name'])
    key = i['name']
    instruments_dict[key] = {
        k: i[k] for k in key_i
    }

In [21]:
instruments_dict

{'CHINAH_HKD': {'name': 'CHINAH_HKD',
  'type': 'CFD',
  'displayName': 'China H Shares',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'AU200_AUD': {'name': 'AU200_AUD',
  'type': 'CFD',
  'displayName': 'Australia 200',
  'pipLocation': 0,
  'displayPrecision': 1,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.05'},
 'USD_PLN': {'name': 'USD_PLN',
  'type': 'CURRENCY',
  'displayName': 'USD/PLN',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'GBP_AUD': {'name': 'GBP_AUD',
  'type': 'CURRENCY',
  'displayName': 'GBP/AUD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'SG30_SGD': {'name': 'SG30_SGD',
  'type': 'CFD',
  'displayName': 'Singapore 30',
  'pipLocation': -1,
  'displayPrecision': 2,
  'tradeUnitsPrecision': 1,
  'marginRate': '0.1'},
 'EUR_USD': {'name': 'EUR_USD',
  'type': 'CURRENCY',
  'displayName': 'EUR/USD',
  

In [22]:
# fetching particular instrument
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.0333'}

In [23]:
with open("Data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

## Fetching a candle pair

In [24]:
def fetch_candles(api_url, pair_name, count="10", granularity="H1"):
    url = f"{api_url}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()
    
    status = response.status_code
    
    if status == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    
    return status, data

In [25]:
status, data = fetch_candles(OANDA_URL, "EUR_USD", count=20)

In [26]:
status

200

In [27]:
len(data)

20

In [28]:
data

[{'complete': True,
  'volume': 8116,
  'time': '2022-07-19T12:00:00.000000000Z',
  'bid': {'o': '1.02528', 'h': '1.02536', 'l': '1.02240', 'c': '1.02280'},
  'mid': {'o': '1.02535', 'h': '1.02543', 'l': '1.02248', 'c': '1.02288'},
  'ask': {'o': '1.02542', 'h': '1.02550', 'l': '1.02255', 'c': '1.02297'}},
 {'complete': True,
  'volume': 9162,
  'time': '2022-07-19T13:00:00.000000000Z',
  'bid': {'o': '1.02278', 'h': '1.02560', 'l': '1.02277', 'c': '1.02407'},
  'mid': {'o': '1.02286', 'h': '1.02568', 'l': '1.02286', 'c': '1.02415'},
  'ask': {'o': '1.02293', 'h': '1.02576', 'l': '1.02293', 'c': '1.02423'}},
 {'complete': True,
  'volume': 12796,
  'time': '2022-07-19T14:00:00.000000000Z',
  'bid': {'o': '1.02408', 'h': '1.02634', 'l': '1.02282', 'c': '1.02450'},
  'mid': {'o': '1.02415', 'h': '1.02648', 'l': '1.02290', 'c': '1.02456'},
  'ask': {'o': '1.02422', 'h': '1.02661', 'l': '1.02297', 'c': '1.02463'}},
 {'complete': True,
  'volume': 7258,
  'time': '2022-07-19T15:00:00.000000

In [39]:
# getting the prices from the JSON object
prices = ['mid', 'bid', 'ask']
ohlc = ['o', 'h', 'l', 'c']
# loop through the prices and make for each one a column key 
# that looks like mid_c, mid_o, mid_h, mid_c
for p in prices:
    for o in ohlc:
        print(f"{p}_{o}")

mid_o
mid_h
mid_l
mid_c
bid_o
bid_h
bid_l
bid_c
ask_o
ask_h
ask_l
ask_c


In [43]:
# collecting certain info from the instruments
# and converting it to a data frame
final_data = []
for candle in data:
    if candle['complete'] == False:
        continue
    new_dict = {}
    new_dict['time'] = parser.parse(candle['time'])
    new_dict['volume'] = candle['volume']
    # loop through the prices and make for each one a column key 
    # that looks like mid_c, mid_o, mid_h, mid_c
    for p in prices:
        for o in ohlc:
            new_dict[f"{p}_{o}"] = float(candle[p][o])
    final_data.append(new_dict)
df = pd.DataFrame.from_dict(final_data)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype                  
---  ------  --------------  -----                  
 0   time    19 non-null     datetime64[ns, tzutc()]
 1   volume  19 non-null     int64                  
 2   mid_o   19 non-null     float64                
 3   mid_h   19 non-null     float64                
 4   mid_l   19 non-null     float64                
 5   mid_c   19 non-null     float64                
 6   bid_o   19 non-null     float64                
 7   bid_h   19 non-null     float64                
 8   bid_l   19 non-null     float64                
 9   bid_c   19 non-null     float64                
 10  ask_o   19 non-null     float64                
 11  ask_h   19 non-null     float64                
 12  ask_l   19 non-null     float64                
 13  ask_c   19 non-null     float64                
dtypes: datetime64[ns, tzutc()](1), float64(12), 

In [45]:
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-07-19 12:00:00+00:00,8116,1.02535,1.02543,1.02248,1.02288,1.02528,1.02536,1.02240,1.02280,1.02542,1.02550,1.02255,1.02297
1,2022-07-19 13:00:00+00:00,9162,1.02286,1.02568,1.02286,1.02415,1.02278,1.02560,1.02277,1.02407,1.02293,1.02576,1.02293,1.02423
2,2022-07-19 14:00:00+00:00,12796,1.02415,1.02648,1.02290,1.02456,1.02408,1.02634,1.02282,1.02450,1.02422,1.02661,1.02297,1.02463
3,2022-07-19 15:00:00+00:00,7258,1.02458,1.02550,1.02312,1.02388,1.02451,1.02543,1.02304,1.02381,1.02464,1.02558,1.02320,1.02395
4,2022-07-19 16:00:00+00:00,4424,1.02388,1.02452,1.02298,1.02404,1.02380,1.02445,1.02291,1.02397,1.02395,1.02460,1.02305,1.02411
5,2022-07-19 17:00:00+00:00,3402,1.02405,1.02441,1.02348,1.02353,1.02397,1.02433,1.02341,1.02345,1.02413,1.02450,1.02356,1.02361
6,2022-07-19 18:00:00+00:00,3665,1.02353,1.02366,1.02258,1.02320,1.02346,1.02359,1.02250,1.02314,1.02360,1.02373,1.02266,1.02327
7,2022-07-19 19:00:00+00:00,5434,1.02317,1.02320,1.02213,1.02215,1.02310,1.02313,1.02205,1.02208,1.02324,1.02327,1.02220,1.02222
8,2022-07-19 20:00:00+00:00,1743,1.02217,1.02276,1.02217,1.02258,1.02210,1.02268,1.02209,1.02248,1.02224,1.02284,1.02224,1.02269
9,2022-07-19 21:00:00+00:00,1009,1.02275,1.02367,1.02265,1.02284,1.02231,1.02317,1.02230,1.02266,1.02319,1.02417,1.02294,1.02303
